In [27]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# the problem with fuzz iis that it does not capture sematic meaning -> good ratio can be very bad since wording is key in TORs
str1 = 'Oracle             database'
str2 = 'Oracle database'
fuzz.token_sort_ratio(str1, str2)

100

In [17]:
# test using sentence models 
from sentence_transformers import SentenceTransformer, util

# load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the two sentences
sentence1 = 'I want to really eat some ice cream at the store'
sentence2 = 'I want to really not eat some ice cream at the store'

# Generate embeddings for each sentence
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Compute cosine similarity between the embeddings
cosine_score = util.pytorch_cos_sim(embedding1, embedding2)
print("Cosine similarity:", cosine_score.item())

Cosine similarity: 0.8974542617797852
